In [4]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import defaultdict
from IPython.display import display
from fuzzywuzzy import fuzz, process
from ITUtils import country_conflicts_finder  # Assuming you have this util function
print('doing something')
# === CONFIG ===
tpafile = './databases/TPAtable.csv'
tablesfolder = 'countriestables'
satnamesfolder = 'satellitenames'
outfolder = 'adm_conflicts'
countrieslistfile = 'countrieslist.csv'

# Create output folder if needed
if not os.path.exists(outfolder):
    os.makedirs(outfolder)

# Load country codes
with open(countrieslistfile, 'r') as f:
    countries = f.read().strip().split(', ')
# # comment this for custom countries list
countries = ['D'] 

# Load reference table
refdf = pd.read_csv(tpafile)

# === PROCESS EACH COUNTRY ===
for ccode in countries:
    print(f"\n=== Processing {ccode} ===")

    # Make folder for this country
    country_outfolder = os.path.join(outfolder, ccode)
    os.makedirs(country_outfolder, exist_ok=True)

    # --- Step 1: Find conflicts ---
    noinfo = country_conflicts_finder(ccode, refdf, tablesfolder, satnamesfolder, country_outfolder)
    if not noinfo:
        continue
    # Save list of satellites with missing carrier frequency info
    with open(os.path.join(country_outfolder, 'noinfooncarrierfrequency.txt'), 'w') as f:
        f.write('The following list contains all the satellite names for which no information on the carrier frequency was found in the ITU database for at least one of the entries.\n')
        f.write('It is possible that the same satellite has other entries with all the required information for channel overlap comparisons.\n\n')
        f.write(str(noinfo))

    # --- Step 2: Create summary table ---
    folder = os.path.join(country_outfolder, 'output_tables')
    all_files = os.listdir(folder)
    worstcase_files = [f for f in all_files if 'worstcase' in f.lower() and 'R_E' not in f]

    fmins, fmaxs, f0s, bws, types, dataframes, satnames_lists = [], [], [], [], [], [], []

    for ff in worstcase_files:
        match = re.search(r'(\d+\.\d+)-(\d+\.\d+)_([A-Z]_[A-Z])', ff)
        if match:
            fmin = float(match.group(1))
            fmax = float(match.group(2))
            f0 = (fmin + fmax) / 2
            bw = round(1e6 * (fmax - fmin))
            ctype = str(match.group(3))
            df = pd.read_csv(os.path.join(folder, ff), low_memory=False)
            satnames = df[' com_el.sat_name'].unique()

            fmins.append(fmin)
            fmaxs.append(fmax)
            f0s.append(f0)
            bws.append(bw)
            types.append(ctype)
            dataframes.append(df)
            satnames_lists.append(satnames)

    summary = []
    for f0_, bw_, ctype, sats in zip(f0s, bws, types, satnames_lists):
        row = {
            'f0': f0_,
            'bandwidth': bw_,
            'conflict_type': ctype,
            'satellite_names': ', '.join(sats),
            'satellite_count': len(sats)
        }
        summary.append(row)

    df_summary = pd.DataFrame(summary)
    df_summary.to_csv(os.path.join(country_outfolder, 'conflict_summary_by_freq_type.csv'), index=False)

    # --- Step 3: Plot summary ---
    records = []
    for f0_, bw_, ctype, sats in zip(f0s, bws, types, satnames_lists):
        if ctype == 'R_E':
            continue
        label = f"{f0_:.3f} MHz / {bw_//1000} kHz"
        records.append({
            'freq_band': label,
            'conflict_type': ctype,
            'satellite_count': len(sats)
        })

    df_plot = pd.DataFrame(records)

    if not df_plot.empty:
        pivot = df_plot.pivot_table(
            index='freq_band',
            columns='conflict_type',
            values='satellite_count',
            aggfunc='sum',
            fill_value=0
        )

        ax = pivot.plot(kind='bar', stacked=True, figsize=(12, 6), colormap='tab20')
        plt.ylabel('Number of satellites')
        plt.xlabel('Frequency / Bandwidth')
        plt.title(f'Conflict types by frequency band for {ccode} (excluding R_E)')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()

        plot_path = os.path.join(country_outfolder, 'conflict_type_barplot.png')
        plt.savefig(plot_path, dpi=300)
        plt.close()
        print(f"✅ Plot saved to: {plot_path}")

    # --- Step 4: Create pivot table (satellite vs worst case percent) ---
    sat_conflict_map = defaultdict(lambda: defaultdict(int))

    for ff, fmin, fmax, f0, bw, conflict_type, satnames, df in zip(worstcase_files, fmins, fmaxs, f0s, bws, types, satnames_lists, dataframes):
        col_prefix = ff.split('_')[0]
        col_suffix = conflict_type[0]
        conflict_column = f"{col_prefix}_{col_suffix}"

        if conflict_column not in df.columns:
            print(f"⚠️ Column {conflict_column} not found in {ff}")
            continue

        for satname in satnames:
            satname = satname.strip()
            try:
                val = df[df[' com_el.sat_name'] == satname][conflict_column].values[0]
            except IndexError:
                continue

            if isinstance(val, str) and '%' in val:
                try:
                    percent = int(re.search(r'(\d+)', val).group(1))
                    conflict_label = f"{f0:.3f}_{bw//1000}kHz_{conflict_type}"
                    sat_conflict_map[satname][conflict_label] = max(
                        sat_conflict_map[satname][conflict_label], percent
                    )
                except Exception as e:
                    print(f"⚠️ Could not parse value '{val}' for satellite {satname}: {e}")

    records = []
    for satname, conflicts in sat_conflict_map.items():
        for conflict_label, percent in conflicts.items():
            records.append({
                'sat_name': satname,
                'conflict_type': conflict_label,
                'worst_case_percent': percent
            })

    df_pivot_source = pd.DataFrame(records)

    if df_pivot_source.empty:
        print("⚠️ No data to create pivot table.")
    else:
        pivot_table = df_pivot_source.pivot(
            index='sat_name',
            columns='conflict_type',
            values='worst_case_percent'
        )
        pivot_path = os.path.join(country_outfolder, 'conflict_percent_pivot_table.csv')
        pivot_table.to_csv(pivot_path)
        print(f"✅ Pivot table saved to: {pivot_path}")

print("\n🎯 Done with all countries!")


doing something

=== Processing D ===
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
loading  D.csv
finding conflicts for  D
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


C:\Users\smen851\PycharmProjects\TPA1_ITU_SE_SIEVE\ITUtils.py:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf[['tpaconflicts', 'percentoverlap']] = ddf.parallel_apply(
C:\Users\smen851\PycharmProjects\TPA1_ITU_SE_SIEVE\ITUtils.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_with_nan['tpaconflicts'] = ''
C:\Users\smen851\PycharmProjects\TPA1_ITU_SE_SIEVE\ITUtils.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

file saved to  adm_conflicts\D\conflicts.csv
satellite names for D  : 
 ['LS-4', 'ASIOT_NBIOT22', 'NAMADGI', 'DIAMANT', 'PAX-1', 'AETHER-C', 'AETHER-K', 'BRINDABELLA', 'D-ISIPELE', 'D-LYRG1-1', 'EIS', 'FLAMINGO-1', 'GOSSAMER', 'HIVE', 'NAMADGI', 'ORASAT', 'SUPA-NGSO-1', '']


,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,orbit.nbr_sat_pl,...,carrier_fr.freq_carr,carrier_fr.seq_emiss,carrier_fr.seq_no,srv_area.ctry,srv_area.f_excl_api,channel.bandwidth,channel.freq_min,channel.freq_max,tpaconflicts,percentoverlap
0,100590924,NaN,D,NaN,DFS-1,23.5,RS49,3/11/2000 12:00:00 am,50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
1,100590925,NaN,D,NaN,DFS-2,28.5,RS49,3/11/2000 12:00:00 am,50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
2,109500171,NaN,D,NaN,GENESIS-8,13.0,11.2,12/06/2012 12:00:00 am,50,NaN,...,NaN,NaN,NaN,NaN,NaN,5000000.0,NaN,NaN,,
3,109500171,NaN,D,NaN,GENESIS-8,13.0,11.2,12/06/2012 12:00:00 am,50,NaN,...,NaN,NaN,NaN,NaN,NaN,1200000.0,NaN,NaN,,
4,109500420,NaN,D,NaN,GENESIS-9,63.0,11.2,13/08/2012 12:00:00 am,50,NaN,...,NaN,NaN,NaN,NaN,NaN,5000000.0,NaN,NaN,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
898769,123520002,NaN,D,NaN,GENESIS-20,40.5,9.6,5/01/2023 12:00:00 am,50,NaN,...,NaN,NaN,NaN,NaN,NaN,80000.0,NaN,NaN,,
898770,123520002,NaN,D,NaN,GENESIS-20,40.5,9.6,5/01/2023 12:00:00 am,50,NaN,...,NaN,NaN,NaN,NaN,NaN,1200000.0,NaN,NaN,,
898771,123520002,NaN,D,NaN,GENESIS-20,40.5,9.6,5/01/2023 12:00:00 am,50,NaN,...,NaN,NaN,NaN,NaN,NaN,1200000.0,NaN,NaN,,
898772,123520002,NaN,D,NaN,GENESIS-20,40.5,9.6,5/01/2023 12:00:00 am,50,NaN,...,NaN,NaN,NaN,NaN,NaN,80000.0,NaN,NaN,,


names with no frequency info ['PAX-1']
expanding the conflict columns for D
['TPA1.401.95525-401.96475_R', 'TPA1.401.9501-401.9699_R', 'TPA1.2055.475-2055.725_R', 'TPA1.401.89525-401.90475_R', 'TPA1.2237.0-2238.0_E', 'TPA1.2065.575-2065.825_R', 'TPA1.401.9501-401.9699_E', 'TPA1.401.8901-401.9099_R', 'TPA1.401.95525-401.96475_E', 'TPA1.401.8901-401.9099_E', 'TPA1.401.89525-401.90475_E', 'TPA1.2202.4-2203.4_E']


INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/
['TPA1.401.95525-401.96475_R', 'TPA1.401.9501-401.9699_R', 'TPA1.2055.475-2055.725_R', 'TPA1.401.89525-401.90475_R', 'TPA1.2237.0-2238.0_E', 'TPA1.2065.575-2065.825_R', 'TPA1.401.9501-401.9699_E', 'TPA1.401.8901-401.9099_R', 'TPA1.401.95525-401.96475_E', 'TPA1.401.8901-401.9099_E', 'TPA1.401.89525-401.90475_E', 'TPA1.2202.4-2203.4_E']
table saved to adm_conflicts\D\output_tables\TPA1.401.95525-401.96475_R_E.csv
table saved to adm_conflicts\D\output_tables\TPA1.401.95525-401.96475_R_E_worstcase.csv
table saved to adm_conflicts\D\output_tables\TPA1.401.9501-401.9699_R_E.csv
table saved to adm_conflicts\D\output_tables\TPA1.401.9501-401.9699_R_E_worstcase.csv
table saved to adm_conflicts\D\output_tables\TPA1.2055.475-2055.725_R_R.csv
table saved to adm_conflicts\D\ou